In [ ]:
# Don't change the names
!mkdir msdir
!mkdir input
!mkdir output

In [ ]:
import json
import stimela
import drastic
import pkg_resources
from drastic import natural_keys
from ipywidgets import widgets, interact
from IPython.display import Javascript

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+4)'))

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
DATA = dict()
MSDIR = 'msdir'
recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR)
cab_names = ['simms', 'simulator', 'calibrator', 'wsclean', 'ddfacet', 'cleanmask', 'casa_makemask',
             'casa_clean', 'lwimager', 'casa_tclean', #'cubical',
             'fitstool', 'pybdsm', 'aegean',]#stimela.CAB
def setup():
    for cab_name in cab_names:
        PARAM_FILE_PATH = pkg_resources.resource_filename('stimela', 'cargo/cab/{}'.format(cab_name))
        PARAM_FILE = '{}/parameters.json'.format(PARAM_FILE_PATH)

        with open(PARAM_FILE) as par:
            DATA[cab_name] = json.load(par)
setup()

In [ ]:
recipe_file = widgets.Text(
    value='',
    placeholder='recipe.json',
    description='Recipe File:',
    disabled=False
)
display(recipe_file)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+3)'))

button = widgets.Button(description="LOAD")
button.on_click(run_all)
display(button)

In [ ]:
Im = []
recipe.jobs = []
data = drastic.get_data(recipe_file.value)
cb_cont = widgets.HBox(background_color='red',height='200px',width='100%')
data_keys = data.keys()
data_keys.sort(key=natural_keys)
for dk in data_keys:
    Im.append(widgets.Checkbox(description=dk,value=True,width=50))
cb_cont.children=[i for i in Im]
display(cb_cont)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="RUN ALL")
button.on_click(run_all)
display(button)

In [ ]:
recipe.jobs = []
data = drastic.get_data(recipe_file.value)
for cab_value, cab_params in data.items():
    cab_num = cab_value.split('_')[0]
    cab_func_name = cab_value.split(cab_num + '_')[-1]
    cab_func = getattr(drastic, cab_func_name, None)
    for c in cb_cont.children:
        if c.description == cab_value:
            if c.value:
                cab_func(recipe, num=cab_num, parameters=cab_params)
drastic.run_all(recipe)